### SST OM4p25

In [1]:
# import cartopy
# import cmocean
# import matplotlib.pyplot as plt
# import matplotlib.colors
# import matplotlib.colorbar
import xarray as xr
# import numpy

In [2]:
from dask.distributed import Client

In [3]:
client= Client('140.208.147.178:39769')

In [4]:
%%time
## net series files 300yr OM4p25
OM4p25 = xr.open_zarr('/work/Nkeh.Boh/SST/OM4p25-100yr/Zarrfile/OM4p25.zarr') #100 yr

W25 = xr.open_zarr('/work/Nkeh.Boh/SST/ZarrDataset/WOA25.zarr/')

W25_av  = W25['ptemp'].mean(axis=(0,1)).load()
OM4p25_av = OM4p25['tos'].mean(axis=0).load()

# anomaly = OM4p25_av - W25_av


CPU times: user 271 ms, sys: 47 ms, total: 318 ms
Wall time: 14.7 s


## Plots

In [ ]:
%%time
G25 = xr.open_dataset('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_grid25.zarr')
G5 = xr.open_dataset('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_grid05.zarr')
M25 = xr.open_dataset('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_mask25.zarr')
M5 = xr.open_dataset('/work/Nkeh.Boh/SST/ZarrDataset/Ocean_mask05.zarr')

xq25 = G25.variables['x'][::2,::2]
yq25 = G25.variables['y'][::2,::2]
a25 = G25.variables['area']; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25.variables['mask']; a25 = a25*m25


def stats(ax, area, anomaly, label):
    mn = (anomaly*area).sum()/area.sum()
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() )
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() )
    qmn, qmx = anomaly.min(), anomaly.max()
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')

# Plot Fig

fig = plt.figure(figsize=(10,7))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
# axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)


ax = plt.axes(projection=cartopy.crs.Robinson(central_longitude=-155))

im = ax.pcolormesh(xq25, yq25, anomaly, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a25, anomaly, 'OMp25')
ax.coastlines()
# axes.append(ax)
plt.title('a) OM4p25')

ax = plt.gcf().add_axes((.25,.5,.5,.03))
cb = matplotlib.colorbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,
                                      orientation='horizontal', extend='both')
cb.ax.set_title('SST bias ($^\circ$C)')